In [63]:
import os
import sys
import pathlib

In [64]:
main_path = '/export/corpora/'
clean_audios = 'siwis_database/wav/'
noise_audios = 'musan'

In [67]:
clean_audios_path = os.path.join(main_path, clean_audios)
noise_audios_path = os.path.join(main_path, noise_audios)

In [66]:
separator = '-'
counter = {}
with open('../data/clean_wav.scp', 'w') as file:
    for language in os.listdir(clean_audios_path):
        for folder in os.listdir(os.path.join(clean_audios_path, language)):
            for utt in os.listdir(os.path.join(clean_audios_path, language, folder)):
                tmp = os.path.join(clean_audios_path, language, folder, utt)
                if pathlib.Path(tmp).suffix == '.wav':
                    file.write('{} {}\n'.format(separator.join(utt.replace('.wav', '').split('_')), tmp))
                    if language in counter:
                        counter[language] += 1
                    else:
                        counter[language] = 1

file.close()
print('Detected {} clean audios'.format(sum(counter.values())))
print(counter)

Detected 14744 audios
{'DE': 2903, 'EN': 3771, 'FR': 5332, 'IT': 2738}


In [84]:
counter = {}

with open('../data/noise_wav.scp', 'w') as file:
    for folder in os.listdir(noise_audios_path):
        if os.path.isdir(os.path.join(noise_audios_path, folder)):
            for subfolder in os.listdir(os.path.join(noise_audios_path, folder)):
                if os.path.isdir(os.path.join(noise_audios_path, folder, subfolder)):
                    for utt in os.listdir(os.path.join(noise_audios_path, folder, subfolder)):
                        if utt.endswith('.wav'):
                            file.write('{} {}\n'.format(utt.replace('.wav', ''),
                                                        os.path.join(noise_audios_path, folder, subfolder, utt)))
                            if folder in counter:
                                counter[folder] += 1
                            else:
                                counter[folder] = 1

file.close()
print('Detected {} noise audios'.format(sum(counter.values())))
print(counter)

Detected 2016 noise audios
{'music': 660, 'noise': 930, 'speech': 426}


In [97]:
list_of_speakers = set()
with open('../data/utt2spk.scp', 'w') as file:
    for line in open('../data/clean_wav.scp'):
        utt, path = line.split()
        spk = utt.split('-')[2]
        list_of_speakers.add(spk)
        file.write('{} {}\n'.format(utt, spk))
        
file.close()
print('{} speakers'.format(len(list_of_speakers)))

36 speakers
